In [11]:
import json
import numpy as np
import pandas as pd
from typing import Any, Dict, List

In [2]:
f = open('jobs.json','r')
data = json.load(f)
df = pd.DataFrame.from_dict(data['jobs'])
drop_cols = ['account', 'cluster', 'container', 'comment', 'array','association', 'derived_exit_code', 'exit_code', 'het', 'job_id', 'name', 'mcs', ]
df = df.drop(columns=drop_cols)

In [3]:
df.head()

,allocation_nodes,constraints,time,flags,group,nodes,partition,priority,qos,required,kill_request_user,reservation,state,steps,tres,user,wckey,working_directory
0,1,None,"{'elapsed': 9482, 'eligible': 1672509600, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1788,standard,1417,normal,"{'CPUs': 24, 'memory': 2048}",None,"{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1788'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
1,1,None,"{'elapsed': 9932, 'eligible': 1672531200, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e2281,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}",None,"{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e2281'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
2,1,None,"{'elapsed': 10601, 'eligible': 1672552800, 'en...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1901,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}",None,"{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1901'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
3,1,None,"{'elapsed': 9408, 'eligible': 1672574400, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1403,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}",None,"{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1403'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
4,1,None,"{'elapsed': 10631, 'eligible': 1672617600, 'en...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1893,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}",None,"{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1893'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...


In [4]:
df_res = df['steps'].apply(pd.Series)
#df_res.columns=['first']
df_res.head(10)
df_res_1 = df_res[0].apply(pd.Series)
df_res_1.head(10)
df_res_2 = df_res_1['CPU'].apply(pd.Series)
df_res_2.head(10)
# df_res_3 = df_res_2['requested_frequency'].apply(pd.Series)
# df_res_3.head(10)

C:\Users\232_DETI\AppData\Local\Temp\ipykernel_20332\845000896.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_res = df['steps'].apply(pd.Series)


,requested_frequency,governor,0
0,"{'min': 0, 'max': 0}",[],NaN
1,"{'min': 0, 'max': 0}",[],NaN
2,"{'min': 0, 'max': 0}",[],NaN
3,"{'min': 0, 'max': 0}",[],NaN
4,"{'min': 0, 'max': 0}",[],NaN
5,"{'min': 0, 'max': 0}",[],NaN
6,"{'min': 0, 'max': 0}",[],NaN
7,"{'min': 0, 'max': 0}",[],NaN
8,"{'min': 0, 'max': 0}",[],NaN
9,"{'min': 0, 'max': 0}",[],NaN


In [5]:
#komorka rozpakowujaca job-tres
def clean_str(names: str): #umozliwia cofanie się w nazwach
    sep = '-'
    res = ''
    names = names.split(sep)[:-1]#cofamy się w nazwach tylko o 1 wartosc
    for elem in names[:-1]:
        res += str(elem)+'-'
    res += names[-1]
    return res

def parse_json(df, names):
    print('names: ', names)
    col_names = []
    df_res = df.apply(pd.Series) #moze warto się zastanowic czy nie przejsc z tym na dataframe, operujemy tutaj na pd.Series
    for col_name,col_data in df_res.iteritems():
        if type(col_data[0]) is list or type(col_data[0]) is dict: #jesli lista albo dict to mozemy zejsc na dol, jesli nie to mamy koncowe zmienne
            print(col_name)
            names = names+'-'+str(col_name) #separator uzywany w funkcji to '-', ze względu na fakt, że '_' jest już używany w nazwach zmiennych
            names = parse_json(df_res[col_name], names) #rekursywność
            names = clean_str(names) #czysczenie nazw przy cofaniu się 
            
            
        else:
            names_final.append(names+'-'+str(col_data[0])) # w tym wypadku trzeba uzyc col_data[0] bo tam jest przechowywana inf czy cpu,mem itd.
            return names

global names_final
names_final = []
parse_json(df['tres'], 'tres')
print(names_final)#names final - columns names, resztę wystarczy skonkatenować w else i stworzyć nowy df

names:  tres


C:\Users\232_DETI\AppData\Local\Temp\ipykernel_20332\3126585328.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name,col_data in df_res.iteritems():


allocated
names:  tres-allocated
0
names:  tres-allocated-0


C:\Users\232_DETI\AppData\Local\Temp\ipykernel_20332\3126585328.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name,col_data in df_res.iteritems():


1
names:  tres-allocated-1


...

C:\Users\232_DETI\AppData\Local\Temp\ipykernel_20332\3126585328.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name,col_data in df_res.iteritems():


2
names:  tres-requested-2


C:\Users\232_DETI\AppData\Local\Temp\ipykernel_20332\3126585328.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name,col_data in df_res.iteritems():


3
names:  tres-requested-3
['tres-allocated-0-cpu', 'tres-allocated-1-mem', 'tres-allocated-2-node', 'tres-allocated-3-billing', 'tres-allocated-nan', 'tres-requested-0-cpu', 'tres-requested-1-mem', 'tres-requested-2-node', 'tres-requested-3-billing', 'tres-requested-nan']


C:\Users\232_DETI\AppData\Local\Temp\ipykernel_20332\3126585328.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name,col_data in df_res.iteritems():


In [28]:
#komorka rozpakowujaca steps, trochę trzeba jeszcze dopisać, nie wiem ile wyjątków trzeba będzie jeszcze pokryć
#do zastanowienia się czy jest to możliwe, żeby pokryc wszystkie wyjątki i rozpakować całość steps
def check_empty_list(y): #funkcja zmieniajaca puste listy na NaN
    if type(y) is list:
        if len(y) == 0:
            y = np.nan
        else:
            y=y
    else:
        y=y
    return y

def clean_str(names: str):
    print('xxx', names, 'xxx')
    sep = '-'
    res = ''
    names = names.split(sep)[:-1]
    for elem in names[:-1]:
        res += str(elem)+'-'
    res += names[-1]
    return res

def clean_df(df: pd.DataFrame): #czyszczenie NaNów i aplikowanie zmian [] na NaN
    for column in df.columns:
        if type(df[column][0]) is list:
            df[column] = df[column].apply(lambda y: check_empty_list(y))
    df.dropna(how='all', axis=1, inplace=True)
    return df

def isfinal(col_data) -> bool:
    if isinstance(col_data[0], list) or isinstance(col_data[0], dict):
        return False
    return True

def parse_json(df: pd.DataFrame, names):
    print('names: ', names)
    col_names = []
    df_res = df.apply(pd.Series, dtype='object')
    print(df_res.columns.to_list())
    df_res = clean_df(df_res)
    print(df_res.columns.to_list())
    for col_name,col_data in df_res.items():
        print('ilosc: ', len(list(df_res.axes[1])))
        if type(col_data[0]) is list or type(col_data[0]) is dict:
            next_name = names+'-'+str(col_name)
            print(next_name)
            next_name = parse_json(df_res[col_name], next_name)
            next_name = clean_str(next_name)
            #tutaj program nie dziala, bo return cofa nas do najwyższego poziomu
            #do zrobienia logika zarządzająca returnami tylko wtedy gdy kończą się zmienne w iteritems()
            
        else:
            names_final.append(names)
            

tmp_df = df.iloc[0:100]

global names_final
names_final = []
parse_json(tmp_df['steps'], 'steps')
print(names_final)

names:  steps
[0, 1, 2]
[0, 1, 2]
ilosc:  3
steps-0
names:  steps-0
['nodes', 'tres', 'time', 'exit_code', 'tasks', 'pid', 'CPU', 'kill_request_user', 'state', 'statistics', 'step', 'task']
['nodes', 'tres', 'time', 'exit_code', 'tasks', 'CPU', 'state', 'statistics', 'step', 'task']
ilosc:  10
steps-0-nodes
names:  steps-0-nodes
['list', 'count', 'range']
['list', 'count', 'range']
ilosc:  3
steps-0-nodes-list
names:  steps-0-nodes-list
[0]
[0]
ilosc:  1
xxx None xxx


AttributeError: 'NoneType' object has no attribute 'split'

In [29]:
type(tmp_df['steps'])

pandas.core.series.Series

In [21]:
def not_dict_or_list(object: Any) -> bool:
    '''
    Return True if `object` is not dict or list.
    '''
    return type(object) is not list and type(object) is not dict

def check_if_terminal(object: Any) -> bool:
    ''' 
    Return True if `object` seems like its terminal variable. 
    Assumption: terminals are any objects that aren't of type 
    list or dict and their length is 0.
    '''
    if type(object) is list and len(object) > 0:
        return False
    
    if type(object) is dict and len(object.keys()) > 0:
        return False
    return True
    
def DFS_unravel_list(lst: List[Any], prefix_name: str) -> List[str]:
    '''
    DFS search for populating col_names global list. 
    '''
    colnames = []
    # if list is terminal then do not expand further. e.g. list of nodes ['e123', 'e324']
    if all([check_if_terminal(x) for x in lst]):
        colnames.append(prefix_name) 
    else:
        for index, value in enumerate(lst):
            # Create next name in standardized format
            new_name = prefix_name + '-#' + str(index)
            # If list then unravel as list
            if type(value) is list:
                colnames += DFS_unravel_list(value, new_name)
            # If dict unravel as dict
            elif type(value) is dict:
                colnames += DFS_unravel_dict(value, new_name)
            # Should not end up here, but for just in case will print error
            else:
                print('LIST???')
    return colnames
                 
def DFS_unravel_dict(dic: Dict[str, Any], prefix_name: str) -> List[str]:
    ''' 
    DFS search for populating col_names global list. 
    Pass dictionary to be mapped from nested form 
    into standatd dash and hashtag separated format.
    '''
    colnames = []
    for index, value in dic.items():
        new_name = prefix_name + '-' + str(index)
        if type(value) is list:
            colnames += DFS_unravel_list(value, new_name)
        elif type(value) is dict:
            colnames += DFS_unravel_dict(value, new_name)
        else:
            if not_dict_or_list(value):
                colnames.append(new_name)
            else:
                print('DICT???')
    return colnames

def get_mapping_for_dict(dictionary: Dict[str, Any], col_name: str) -> List[str]:
    colnames = DFS_unravel_dict(dictionary, col_name)
    return colnames
    

# GET MAPPING only looking at first row of data
df_col = df['steps'].iloc[0:1000].apply(lambda x: x[0]) 
mapping_list = get_mapping_for_dict(df_col.iloc[1], 'steps')
print(len(mapping_list))
print(mapping_list[0:5])
print(df_col.shape)

201
['steps-nodes-list', 'steps-nodes-count', 'steps-nodes-range', 'steps-tres-requested-max-#0-type', 'steps-tres-requested-max-#0-name']
(1000,)


In [22]:
def extract_by_map_str(dictionary: Dict[str, Any], map_string: str) -> Any:
    ''' 
    Extract value from dictionary by following path specified by mapping string.
    If path cannot be reached then returned value should be np.nan.
    '''
    map_steps = map_string.split('-')
    # extract value, first name in split is the overall colname
    value = dictionary
    for step in map_steps[1:]:
        if type(value) is list and step[0] == '#': # hashtag means that we map to the list index 
            index = int(step[1:]) 
            # If index is not in list return np.nan
            if len(value) <= index:
                return np.nan
            # Else proceed further with the list
            else:
                value = value[index]
        else:
            if type(value) is not dict or step not in value.keys():
                return np.nan
            else:
                value = value[step] # go into dictionary field

    return value

def flatten_series_by_mapping(series: pd.Series, mapping: List[str]) -> pd.DataFrame:
    flat = list()
    # Iterative approach - probably slow, but should be sufficent for now
    for row_idx, dictionary in series.items():
        row = dict()
        for map_string in mapping:
            row[map_string] = extract_by_map_str(dictionary, map_string)
        flat.append(row)
    return pd.DataFrame(flat)


flatten_series_by_mapping(df_col, mapping_list)

,steps-nodes-list,steps-nodes-count,steps-nodes-range,steps-tres-requested-max-#0-type,steps-tres-requested-max-#0-name,steps-tres-requested-max-#0-id,steps-tres-requested-max-#0-count,steps-tres-requested-max-#0-task,steps-tres-requested-max-#0-node,steps-tres-requested-max-#1-type,...,steps-CPU-governor,steps-kill_request_user,steps-state,steps-statistics-CPU-actual_frequency,steps-statistics-energy-consumed,steps-step-job_id,steps-step-het-component,steps-step-id,steps-step-name,steps-task-distribution
0,[e1788],1,e1788,cpu,None,1,150077410,0,e1788,mem,...,[],None,COMPLETED,0,0,25977766,None,batch,batch,Unknown
1,[e2281],1,e2281,cpu,None,1,155892610,0,e2281,mem,...,[],None,COMPLETED,0,0,26081263,None,batch,batch,Unknown
2,[e1901],1,e1901,cpu,None,1,180730980,0,e1901,mem,...,[],None,COMPLETED,0,0,26081264,None,batch,batch,Unknown
3,[e1403],1,e1403,cpu,None,1,153234590,0,e1403,mem,...,[],None,COMPLETED,0,0,26081265,None,batch,batch,Unknown
4,[e1893],1,e1893,cpu,None,1,181754220,0,e1893,mem,...,[],None,COMPLETED,0,0,26081267,None,batch,batch,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,[e1726],1,e1726,cpu,None,1,1787093060,0,e1726,mem,...,[],None,COMPLETED,0,0,27211012,None,batch,batch,Unknown
996,[e1231],1,e1231,cpu,None,1,9911090,0,e1231,mem,...,[],None,COMPLETED,0,0,27211013,None,batch,batch,Unknown
997,[e1328],1,e1328,cpu,None,1,9768440,0,e1328,mem,...,[],None,COMPLETED,0,0,27211014,None,batch,batch,Unknown
998,[e1442],1,e1442,cpu,None,1,904298620,0,e1442,mem,...,[],None,COMPLETED,0,0,27211015,None,batch,batch,Unknown


Na pale

In [7]:
#concat na pale
for (columnName, columnData) in df.iteritems():
    print('Column Name : ', columnName)
    # print('Column Data : ', columnData)
    df = pd.concat([df, columnData.apply(pd.Series)], axis=0)
    print(df.columns)
#     print('Column Contents : ', columnData.values)
df.head()

Column Name :  comment
Index(['comment', 'allocation_nodes', 'array', 'association', 'cluster',
       'constraints', 'container', 'derived_exit_code', 'time', 'exit_code',
       'flags', 'group', 'het', 'job_id', 'name', 'mcs', 'nodes', 'partition',
       'priority', 'qos', 'required', 'kill_request_user', 'reservation',
       'state', 'steps', 'tres', 'user', 'wckey', 'working_directory',
       'administrator', 'job', 'system'],
      dtype='object')
Column Name :  allocation_nodes
Index([                  0,     'administrator',  'allocation_nodes',
                   'array',       'association',           'cluster',
                 'comment',       'constraints',         'container',
       'derived_exit_code',         'exit_code',             'flags',
                   'group',               'het',               'job',
                  'job_id', 'kill_request_user',               'mcs',
                    'name',             'nodes',         'partition',
                '

Index([                  0,                   1,           'account',
           'administrator',  'allocation_nodes',             'array',
             'association',           'cluster',           'comment',
             'constraints',         'container', 'derived_exit_code',
                 'elapsed',          'eligible',               'end',
               'exit_code',             'flags',             'group',
                     'het',               'job',            'job_id',
       'kill_request_user',             'limit',            'limits',
                     'mcs',              'name',             'nodes',
               'partition',          'priority',               'qos',
                'required',       'reservation',       'return_code',
                   'start',             'state',            'status',
                   'steps',        'submission',         'suspended',
                  'system',              'task',           'task_id',
                    

Index([                  0,                   1,           'account',
           'administrator',  'allocation_nodes',             'array',
             'association',           'cluster',           'comment',
             'constraints',         'container', 'derived_exit_code',
                 'elapsed',          'eligible',               'end',
               'exit_code',             'flags',             'group',
                     'het',               'job',            'job_id',
              'job_offset', 'kill_request_user',             'label',
                   'limit',            'limits',               'mcs',
                    'name',             'nodes',         'partition',
                'priority',               'qos',          'required',
             'reservation',       'return_code',             'start',
                   'state',            'status',             'steps',
              'submission',         'suspended',            'system',
                    

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
               'suspended',            'system',              'task',
                 'task_id',              'time',             'total',
                    'tres',              'user',             'wckey',
       'working_directory'],
      dtype='object', length=561)
Column Name :  tres
Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                    'task',           'task_id',              'time',
                   'total',              'tres',              'user',
                   'wckey', 'working_directory',         'allocated',

,0,1,2,3,4,5,6,7,8,9,...,suspended,system,task,task_id,time,total,tres,user,wckey,working_directory
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'elapsed': 9482, 'eligible': 1672509600, 'end...",NaN,"{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'elapsed': 9932, 'eligible': 1672531200, 'end...",NaN,"{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'elapsed': 10601, 'eligible': 1672552800, 'en...",NaN,"{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'elapsed': 9408, 'eligible': 1672574400, 'end...",NaN,"{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'elapsed': 10631, 'eligible': 1672617600, 'en...",NaN,"{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
